In [1152]:
import pandas as pd
import numpy as np
import pylab 
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta
import os, sys

### Instruction
Read your subset data and subscribers list (may change a little bit later) here and you will get two csv files in the same directory for weekly and monthly churn rate. It's the only place you need to do mannually. 

In [1153]:
# df_subscribers = pd.read_excel('../data/HY_version171819_threshold7_with1st purchase.xlsx','FINAL')

In [1154]:
# df_subscribers['first_pur'] = pd.to_datetime(df_subscribers['first_pur']).dt.strftime("%Y-%m-%d")
# df_subscribers['last_pur'] = pd.to_datetime(df_subscribers['last_pur']).dt.strftime("%Y-%m-%d")

In [1155]:
# You need to change yourself
subcategory_name = 'GERIATRICS_sub_final' 

In [1156]:
df_sales = pd.read_csv("../data/Final subscriber/"+ subcategory_name + ".csv")

In [1164]:
# df_sales = pd.read_excel('../data/HY_version171819_threshold7_with1st purchase.xlsx','FINAL')

## Change Subcategory Here

In [1161]:
# df_sales = df_sales[df_sales['Animal_Species'] == 'DOG']

KeyError: 'Animal_Species'

In [1115]:
df_sales['first_pur'] = pd.to_datetime(df_sales['first_pur']).dt.strftime("%Y-%m-%d")
df_sales['last_pur'] = pd.to_datetime(df_sales['last_pur']).dt.strftime("%Y-%m-%d")

In [1116]:
df_sales = df_sales.sort_values(by=['first_pur'])

In [1117]:
df_sales_171819 = df_sales.loc[(df_sales['first_pur']!='2018-12-12')]

### Build the start_end table

In [1118]:
df_start_end = df_sales_171819[['index','first_pur','last_pur','avg_interval']]

In [1119]:
def f(x):    
    return pd.to_datetime(x['last_pur']) +  timedelta(days=int(x['avg_interval']))
df_start_end['subs_churn'] = df_start_end.apply(f, axis=1)

/Users/renmengyuan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


## Churn Rates

**<center>Churn Rate = Customers Left / (Average Existing Customers)</center>** <br/>

#### daily churn

Let's first create a new dataframe with each day from 2017-1-1 to 2020-2-10 as the key column 

In [1120]:
# Getting number of days need to fill the column
from datetime import date
d0 = date(2017, 1, 1)
d1 = date(2020, 2, 10)
delta = d1 - d0

In [1121]:
# create an array of dates starting at '2017-01-01', one per day. and set day as index
rng = pd.date_range('2017-01-01', periods=1135, freq='D')
df_calender = pd.DataFrame({ 'Date': rng}) 

In [1122]:
df_calender['Date']=pd.to_datetime(df_calender['Date']).dt.strftime("%Y-%m-%d")

In [1123]:
df_calender_2 = df_calender[:]

new_subscriber fill

In [1124]:
df_start_end.reset_index(inplace = True)

In [1125]:
df_start_end = df_start_end.drop(columns = ['level_0'])

In [1126]:
i = 0
for index, row in df_calender_2.iterrows():
    count = 0
    while i < df_start_end.shape[0] and df_start_end.at[i,'first_pur'] == row['Date']:
        count += 1
        i += 1
    df_calender_2.at[index,'new_subs'] = count

/Users/renmengyuan/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/renmengyuan/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [1127]:
df_calender_2['new_subs'].max()

36.0

In [1128]:
df_sort_by_date_end = df_start_end.sort_values(by=['subs_churn'])

In [1129]:
df_sort_by_date_end.reset_index(inplace = True)

In [1130]:
df_sort_by_date_end['subs_churn']=pd.to_datetime(df_sort_by_date_end['subs_churn']).dt.strftime("%Y-%m-%d")

In [1131]:
i = 0
for index, row in df_calender_2.iterrows():
    count = 0
    while i < df_sort_by_date_end.shape[0] and df_sort_by_date_end.at[i,'subs_churn'] == row['Date']:
        count += 1
        i += 1
    df_calender_2.at[index,'new_subs_churn'] = count

/Users/renmengyuan/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/renmengyuan/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [1132]:
df_calender_2['new_subs_churn'].max()

32.0

#### Now calculate the daily existing customer column
The first one or two months existing subscribers base may not be very accurate due to ignorance of active subscribers coming from the end of 2016. This error will gradually disappear after two months or so. Let's first regard the number of unique subscribers during the first month of 2017 as existing subscribers.

In [1133]:
df_sales_week_initial = df_sales_171819.loc[(df_sales_171819['first_pur'] < '2017-01-08') ]

In [1134]:
df_sales_month_initial = df_sales_171819.loc[(df_sales_171819['first_pur'] < '2017-02-01')]

In [1135]:
existing_week_initial = df_sales_week_initial['index'].nunique()

In [1136]:
existing_month_initial = df_sales_month_initial['index'].nunique()

In [1137]:
df_existing_week = df_calender_2[:]

In [1138]:
df_existing_month = df_calender_2[:]

In [1139]:
df_existing_week.at[0,'existing_subscribers'] = existing_week_initial

In [1140]:
df_existing_month.at[0,'existing_subscribers'] = existing_month_initial

In [1141]:
for i in range(1,df_existing_week.shape[0]):
    df_existing_week.at[i,'existing_subscribers'] = df_existing_week.at[i-1,'existing_subscribers'] \
    + df_existing_week.at[i-1,'new_subs'] - df_existing_week.at[i-1,'new_subs_churn']

In [1142]:
for i in range(1,df_existing_month.shape[0]):
    df_existing_month.at[i,'existing_subscribers'] = df_existing_month.at[i-1,'existing_subscribers'] \
    + df_existing_month.at[i-1,'new_subs'] - df_existing_month.at[i-1,'new_subs_churn']

#### Monthly/Weekly churn

<img src="churn_rate.png" width=500 height=500 />

https://www.profitwell.com/blog/the-complete-saas-guide-to-calculating-churn-rate-and-keeping-it-simple

In [1143]:
df_existing_week['Date'] = pd.to_datetime(df_existing_week['Date'], errors='coerce')
df_existing_week['Week'] = df_existing_week['Date'].dt.strftime('%Y-%U')

/Users/renmengyuan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/renmengyuan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [1144]:
df_existing_month['Month'] = pd.to_datetime(df_existing_month['Date']).dt.to_period('M')

/Users/renmengyuan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


### Calculate monthly/weekly churn rate is a new and more precise way
<img src="shopify_way.png" width=500 height=500 />
https://www.profitwell.com/blog/the-complete-saas-guide-to-calculating-churn-rate-and-keeping-it-simple

In [1145]:
df_month_exist = df_existing_month.groupby(['Month']).agg({'new_subs_churn':'sum','existing_subscribers':'mean','new_subs':'sum'})

In [1146]:
df_month_exist['monthly_subs_churn_rate'] = df_month_exist['new_subs_churn']/df_month_exist['existing_subscribers'] 

In [1147]:
df_month_exist = df_month_exist.fillna(0)

In [1148]:
df_week_exist = df_existing_week.groupby(['Week']).agg({'new_subs_churn':'sum','existing_subscribers':'mean','new_subs':'sum'})

In [1149]:
df_week_exist['weekly_subs_churn_rate'] = df_week_exist['new_subs_churn']/df_week_exist['existing_subscribers'] 

In [1150]:
df_week_exist = df_week_exist.fillna(0)

In [1151]:
dfs = {subcategory_name + "_monthly_subs_churn_rate":df_month_exist, subcategory_name + "_weekly_subs_churn_rate":df_week_exist} 

for k, v in dfs.items():
    path_name = '../output/subcategory_churn_rate/subscribers/' 
    if not os.path.exists(path_name):
        os.makedirs(path_name)
    v.to_csv(path_name + '{}.csv'.format(k))